# <center> Testing torch-serve FuSA </center>

Este notebook apunta a documentar cómo enviar archivos de audio wav a la API de TorchServe FuSA para hacer inferencias.

Contacto: Matthieu Vernier (mvernier@inf.uach.cl), Victor Várgas (victorvargassandoval93@gmail.com)

# 1. Cargar un archivo de audio local

In [1]:
import os
folder_results = 'data/'
audiofile = 'dogs.mpeg'
audiofile_path = f"{folder_results}{audiofile}"
audio_format = audiofile.split('.')[1]
#Si el formato no es wav, realizar conversión
from pydub import AudioSegment
if audio_format != 'wav':
    result_wav_filename = f"{folder_results}demo.wav"
    track = AudioSegment.from_file(audiofile_path)
    file_handle = track.export(result_wav_filename, format='wav')
    audiofile_path = result_wav_filename

In [2]:
#Escuchar el archivo
import IPython.display as ipd
ipd.Audio(audiofile_path)

# 2. Datos para conectarse con torch-serve

In [3]:
INFERENCE_ADDRESS = 'http://45.79.145.155:8080'
MANAGEMENT_ADDRESS = 'http://45.79.145.155:8081'

# 3. Estado del servidor y lista de modelos disponibles

In [4]:
import requests
import pprint
#Obtiene el estado del servidor
ping = requests.get(INFERENCE_ADDRESS + '/ping')
print(f'Código respuesta: {ping.status_code}')
ping.json()

Código respuesta: 200


{'status': 'Healthy'}

In [5]:
#Obtiene el estado del servidor
models = requests.get(MANAGEMENT_ADDRESS + '/models')
print(f'Código respuesta: {models.status_code}')

#Lista de modelos
models.json()

Código respuesta: 200


{'models': [{'modelName': 'fusa-naive', 'modelUrl': 'fusa-naive.mar'},
  {'modelName': 'fusa-pann', 'modelUrl': 'fusa-pann.mar'}]}

# 4. Inferencia de modelos en torch-serve

## 4.1. Modelo fusa-naive

In [6]:
#Información modelo fusa-naive
fusa_naive = requests.get(MANAGEMENT_ADDRESS + '/models/fusa-naive')
display(fusa_naive.json())

[{'modelName': 'fusa-naive',
  'modelVersion': '0.1',
  'modelUrl': 'fusa-naive.mar',
  'runtime': 'python',
  'minWorkers': 1,
  'maxWorkers': 1,
  'batchSize': 3,
  'maxBatchDelay': 5000,
  'loadedAtStartup': True,
  'workers': [{'id': '9000',
    'startTime': '2021-10-20T03:00:22.995Z',
    'status': 'READY',
    'memoryUsage': 158986240,
    'pid': 57,
    'gpu': False,
    'gpuUsage': 'N/A'}]}]

In [8]:
#Obtiene una predicción modelo fusa-naive
import time
start = time.time()

files = {'data': open(audiofile_path,'rb')}
prediction = requests.post(
    INFERENCE_ADDRESS + '/predictions/fusa-naive', files=files)
print(f'Código respuesta: {prediction.status_code}')
print(f'{time.time() - start} segundos')
prediction.json()

Código respuesta: 200
6.632507801055908 segundos


{'animal/dog': 0.708381175994873,
 'mechanical/explosives': 0.12466676533222198,
 'animal/others': 0.07243522256612778,
 'humans/others': 0.07062073051929474,
 'animal/bird': 0.010865133255720139,
 'environmental/others': 0.010082419030368328,
 'alerts/siren': 0.0015191191341727972,
 'mechanical/others': 0.0012114327400922775,
 'vehicles/airborne': 0.00011097479000454769,
 'environmental/wind': 5.297106326906942e-05,
 'mechanical/cutting': 4.406927837408148e-05,
 'alerts/horns': 5.469109964906238e-06,
 'vehicles/others': 3.0601520393247483e-06,
 'environmental/rain': 1.2905111361760646e-06,
 'alerts/bells': 1.2938983218191424e-07}

## 4.2. Modelo fusa-pann

In [9]:
#Información modelo fusa-pann
fusa_pann = requests.get(MANAGEMENT_ADDRESS + '/models/fusa-pann')
display(fusa_pann.json())

[{'modelName': 'fusa-pann',
  'modelVersion': '0.2',
  'modelUrl': 'fusa-pann.mar',
  'runtime': 'python',
  'minWorkers': 1,
  'maxWorkers': 1,
  'batchSize': 1,
  'maxBatchDelay': 100,
  'loadedAtStartup': True,
  'workers': [{'id': '9001',
    'startTime': '2021-10-20T03:01:22.611Z',
    'status': 'READY',
    'memoryUsage': 643760128,
    'pid': 97,
    'gpu': False,
    'gpuUsage': 'N/A'}]}]

In [10]:
#Obtiene una predicción modelo fusa-pann
import time
start = time.time()

files = {'data': open(audiofile_path,'rb')}
prediction = requests.post(
    INFERENCE_ADDRESS + '/predictions/fusa-pann', files=files)
print(f'Código respuesta: {prediction.status_code}')
print(f'{time.time() - start} segundos')
prediction.json()

Código respuesta: 200
2.835015058517456 segundos


{'animal/dog': 1.0,
 'animal/others': 2.8016179953738174e-07,
 'vehicles/airborne': 2.1428739671591757e-07,
 'animal/bird': 2.0313358106704982e-07,
 'environmental/wind': 1.2393562087709142e-07,
 'mechanical/cutting': 1.5631792749104534e-08,
 'humans/others': 1.5270286146673584e-09,
 'alerts/horns': 4.470087841546011e-11,
 'vehicles/others': 1.0918833545647821e-11,
 'alerts/bells': 6.5385019447361525e-12,
 'environmental/rain': 2.0644200324909656e-12,
 'mechanical/others': 9.116899943317769e-13,
 'alerts/siren': 3.349796568602459e-13,
 'environmental/others': 7.552864041658613e-15,
 'mechanical/explosives': 9.543122111228992e-16}